In [1]:
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt
import tqdm


In [2]:

# Generated by GPT o3-mini
def get_flame_temperature(fuel_composition, oxidizer_composition, phi, initial_temperature, initial_pressure,
                            combustion_mechanism="gri30.yaml"):
    gas = ct.Solution(combustion_mechanism) # Load the GRI-Mech 3.0 mechanism.
    gas.set_equivalence_ratio(phi, fuel_composition, oxidizer_composition) # Set the fuel-oxidizer mixture using the specified equivalence ratio. The fuel and oxidizer strings include all the species in the fuel and oxidizer gas.
    gas.TP = initial_temperature, initial_pressure # Set the initial state of the gas using temperature and pressure.
    gas.equilibrate('HP') # Equilibrate the gas under constant enthalpy and pressure conditions.
    return gas.T # adiabatic flame temperature

def get_laminar_flame_speed(fuel_composition, oxidizer_composition, phi, initial_temperature, initial_pressure,
                                combustion_mechanism="gri30.yaml"):
    gas = ct.Solution(combustion_mechanism) # Load the GRI-Mech 3.0 mechanism.
    gas.TP = initial_temperature, initial_pressure # Set the initial state of the gas using temperature and pressure.
    gas.set_equivalence_ratio(phi, fuel_composition, oxidizer_composition) # Set the fuel-oxidizer mixture using the specified equivalence ratio. The fuel and oxidizer strings include all the species in the fuel and oxidizer gas.
    #gas.equilibrate('HP') # Equilibrate the gas under constant enthalpy and pressure conditions.
    domain_width = 0.03 # Specify an initial guess for the domain width (in meters) for the flame simulation.
    flame = ct.FreeFlame(gas, width=domain_width)
    flame.set_refine_criteria(ratio=3, slope=0.06, curve=0.12) # Set the grid refinement criteria to accurately capture steep gradients.
    flame.solve(loglevel=0, auto=True) # Solve the flame structure problem. Setting loglevel=1 provides some output during the solution.
    return flame.velocity[0] # For a freely propagating flame, the inlet velocity corresponds to the laminar flame speed (in m/s).

param_dict={
    'combustion_mechanism': "gri30.yaml",
    'fuel_composition': "CH4:96.57, C2H6:0.28, C3H8:0.03, CO2:2.03, N2:1.06, Ar:0.03", # He:0.03 is replaced by Ar:0.03 because He is not supported yet.,
    'oxidizer_composition': "O2:2, N2:7.524", # Oxidizer composition representing air (by mole)
    'initial_temperature': 5.0 + 273.15,
    'initial_pressure': 0.9*ct.one_atm,
    'phi': 1.0
}


'''
print(get_flame_temperature(**{
    'combustion_mechanism': "gri30.yaml",
    'fuel_composition': "CH4:96.57, C2H6:0.28, C3H8:0.03, CO2:2.03, N2:1.06, Ar:0.03", # He:0.03 is replaced by Ar:0.03 because He is not supported yet.,
    'oxidizer_composition': "O2:2, N2:7.524", # Oxidizer composition representing air (by mole)
    'initial_temperature': T_array[0] + 273.15,
    'initial_pressure': P_array[0] * ct.one_atm,
    'phi': Phi_array[0]
}))
'''
#print(get_flame_temperature(**param_dict))
#print(get_laminar_flame_speed(**param_dict))

'\nprint(get_flame_temperature(**{\n    \'combustion_mechanism\': "gri30.yaml",\n    \'fuel_composition\': "CH4:96.57, C2H6:0.28, C3H8:0.03, CO2:2.03, N2:1.06, Ar:0.03", # He:0.03 is replaced by Ar:0.03 because He is not supported yet.,\n    \'oxidizer_composition\': "O2:2, N2:7.524", # Oxidizer composition representing air (by mole)\n    \'initial_temperature\': T_array[0] + 273.15,\n    \'initial_pressure\': P_array[0] * ct.one_atm,\n    \'phi\': Phi_array[0]\n}))\n'

In [3]:
Phi_array = np.array([1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9,
                      2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9,
                      3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
                      4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9,])
T_array = np.array([6,8,10,12,14,16,18,20,22,24,26,28,30,32,34])
P_array = np.array([0.90,0.95,1.00,1.05])


flame_temperature_3D_array = np.zeros((len(Phi_array),len(T_array),len(P_array)))
flame_speed_3D_array = np.zeros((len(Phi_array),len(T_array),len(P_array)))
print(flame_temperature_3D_array.shape)


(40, 15, 4)


In [ ]:
for i, phi in tqdm.tqdm(enumerate(Phi_array)):
    print('Phi = {}'.format(phi))
    for j, t in enumerate(T_array):
        for k, p in enumerate(P_array):
            param_dict['phi'] = phi
            param_dict['initial_temperature'] = t + 273.15
            param_dict['initial_pressure'] = p * ct.one_atm
            flame_temperature_3D_array[i][j][k] = get_flame_temperature(**param_dict)
print("Done.")

In [ ]:
for i, phi in tqdm.tqdm(enumerate(Phi_array)):
    print('Phi = {}'.format(phi))
    for j, t in enumerate(T_array):
        for k, p in enumerate(P_array):
            param_dict['phi'] = phi
            param_dict['initial_temperature'] = t + 273.15
            param_dict['initial_pressure'] = p * ct.one_atm
            flame_speed_3D_array[i][j][k] = get_laminar_flame_speed(**param_dict)
print("Done.")

0it [00:00, ?it/s]

Phi = 1.0


In [ ]:
object_dict = {
    "flame_temperature_3D_array": flame_temperature_3D_array,
    "flame_speed_3D_array": flame_speed_3D_array,
    "Phi_array": Phi_array,
    "T_array": T_array,
    "P_array": P_array,
    "metadata": "Flame temperature and speed values at grid points. \
        Each array has dimension ({}, {}, {}). \
        The values are interpolated from flame simulation using `phi`, `T`, and `P`. \
        The values of `phi`, `T`, `P` grid points are included in the dict.".format(len(Phi_array), len(T_array), len(P_array))
}
import pickle
pickle.dump(object_dict, open("flame_mechanism_20250217.pkl", "wb"))
print("Done.")